In [184]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import random



In [185]:
# load all data
WS_total = pd.read_excel('8124-data.xlsx')
WS_np = np.array(WS_total)
WS_np.shape

(8124, 23)

In [186]:
# create a random data set
cnt = 0
rows = 9
bin_data = np.zeros((rows,rows))
# bin_data.shape
idx = np.random.randint(8123, size=rows)
rnd_rows = WS_np[idx,:]
# rnd_rows.shape
for i in range(rows):
    for j in range(rows):
        cnt = 0
        for k in range(23):
            if(rnd_rows[i][k] == rnd_rows[j][k]):
                cnt += 1
        if(cnt >= 12):
            bin_data[i][j] = 1
            bin_data[j][i] = 1
        else:
            bin_data[i][j] = -1
            bin_data[j][i] = -1
df = pd.DataFrame (bin_data)
fname = 'rnd_' + str(rows) + '_rows.xlsx'
df.to_excel(fname, index=False)
WS2 = pd.read_excel(fname)
WS2_np = np.array(WS2)
WS2_np.shape

(9, 9)

In [187]:
# load a dataset
num_of_original_variables = 9
fname = 'rnd_' + str(num_of_original_variables) + '_rows.xlsx'
WS = pd.read_excel(fname)
WS_np = np.array(WS)
calculated_C = np.empty(0)
num_of_variables = 0
for i in range(num_of_original_variables):
    for j in range(num_of_original_variables):
        if(i<j):
            num_of_variables += 1
            if(WS_np[i][j] == 1):
                calculated_C = np.append(calculated_C,[1])
            else:
                calculated_C = np.append(calculated_C,[-1])                
print("num_of_variables: ",num_of_variables)


num_of_variables:  36


In [188]:
# calculate num_of_triangular_constraints and fill Aub
calculated_Aub = np.empty((0,num_of_variables))
num_of_triangular_constraints = 0
for i in range(num_of_variables):
    for j in range(num_of_variables):
        for k in range(num_of_variables):
            if((i<j) and (j<k)):
                num_of_triangular_constraints += 1
                temp_array = np.zeros(num_of_variables) 
                temp_array[i] = 1
                temp_array[j] = -1
                temp_array[k] = -1
                calculated_Aub = np.append(calculated_Aub,[temp_array],axis=0)
print("num_of_triangular_constraints: ",num_of_triangular_constraints)
print("calculated_Aub.shape: ",calculated_Aub.shape)
# fill Bub with zeroes
temp_array = np.zeros(num_of_triangular_constraints)
calculated_Bub = np.transpose([temp_array])
# adding Bound
required_Bound = np.empty((0,2))
for i in range(num_of_variables):
    required_Bound = np.append(required_Bound,[[0,1]],axis=0)


num_of_triangular_constraints:  7140
calculated_Aub.shape:  (7140, 36)


In [189]:
opt = linprog(c=calculated_C, A_ub=calculated_Aub, b_ub=calculated_Bub,
               A_eq=None, b_eq=None, bounds=required_Bound,
               method="revised simplex")

In [190]:
def fill_lower_diag(a):
    n = int(np.sqrt(len(a)*2))+1
    mask = np.tri(n,dtype=bool, k=-1) # or np.arange(n)[:,None] > np.arange(n)
    out = np.zeros((n,n),dtype=float)
    out[mask] = a
    return out

In [191]:
lp_solution = fill_lower_diag(opt.x)
lp_solution = np.transpose(lp_solution) + lp_solution
print("lp_solution:\n",lp_solution)
print("\nsimilarity:\n",WS_np)

lp_solution:
 [[0.  0.5 0.5 1.  1.  1.  1.  1.  1. ]
 [0.5 0.  0.  1.  0.5 1.  1.  0.5 1. ]
 [0.5 0.  0.  1.  1.  1.  1.  0.5 0.5]
 [1.  1.  1.  0.  1.  1.  1.  0.5 0.5]
 [1.  0.5 1.  1.  0.  0.5 1.  1.  1. ]
 [1.  1.  1.  1.  0.5 0.  1.  1.  0.5]
 [1.  1.  1.  1.  1.  1.  0.  1.  1. ]
 [1.  0.5 0.5 0.5 1.  1.  1.  0.  1. ]
 [1.  1.  0.5 0.5 1.  0.5 1.  1.  0. ]]

similarity:
 [[ 1 -1  1  1 -1 -1 -1 -1  1]
 [-1  1 -1 -1 -1 -1 -1 -1  1]
 [ 1 -1  1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1  1 -1  1  1  1 -1]
 [-1 -1 -1 -1  1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1  1  1  1 -1]
 [-1 -1 -1  1 -1  1  1  1 -1]
 [-1 -1 -1  1 -1  1  1  1 -1]
 [ 1  1 -1 -1 -1 -1 -1 -1  1]]


In [196]:
num_of_iterations = 2000
stack_sol = np.zeros((1,num_of_original_variables + 1))
for z in range(num_of_iterations):
    clusters=[set() for i in range(num_of_original_variables)]
    V_prim=[set() for i in range(2)]
    v_prim_index_1 = 0
    v_prim_index_2 = 1
    for i in range(num_of_original_variables):
        V_prim[v_prim_index_1].add(i)
    current_cluster = [0,0]
    while(1):
    # if(1):
        if(len(V_prim[v_prim_index_1]) == 0):
            break
        pivot = V_prim[v_prim_index_1].pop()
        # print("v_prim_index_1 is: " + str(v_prim_index_1))
        # print(str(pivot) + " choosed as pivot")
        clusters[current_cluster[0]].add(pivot)
        for i in V_prim[v_prim_index_1]:
            rnd_temp = np.random.rand()
            if(rnd_temp > lp_solution[pivot][i]):
                #dont add it to v_prim_index_2
                clusters[current_cluster[0]].add(i)
                # print(str(i) + " added to cluster " + str(current_cluster[0]))
            else:
                rnd_temp = np.random.rand()
                if(rnd_temp > lp_solution[pivot][i]):
                    current_cluster[1] += 1
                    #dont add it to v_prim_index_2
                    clusters[current_cluster[1]].add(i)
                    # print(str(i) + " added to cluster " + str(current_cluster[1]))
                else:
                    #remain 0(V_prim)
                    V_prim[v_prim_index_2].add(i)
                    # print(str(i) + " added to v_prim_" + str(v_prim_index_2))

        current_cluster[0] = current_cluster[1] + 1
        current_cluster[1] = current_cluster[1] + 1
        V_prim[v_prim_index_1].clear()
        if(v_prim_index_1 == 1):
            v_prim_index_1 = 0
            v_prim_index_2 = 1
        else:
            v_prim_index_1 = 1
            v_prim_index_2 = 0
    # print(clusters) 
    final_solution = np.zeros(num_of_original_variables + 1)
    for i in range(num_of_original_variables):
        for j in clusters[i]:
            final_solution[j] = i
    violations = 0
    for i in range(num_of_original_variables):
        for j in range(num_of_original_variables):
            if((final_solution[i] != final_solution[j]) and (WS_np[i][j] == 1)):
                violations += 1
            if((final_solution[i] == final_solution[j]) and (WS_np[i][j] == -1)):
                violations += 1
    final_solution[-1] = violations
    # print("final_solution is: ",final_solution)            
    # print("number of violations is: ",violations)
    final_solution2 = final_solution.reshape((1,-1))
    stack_sol = np.append(stack_sol,final_solution2,axis=0)  

stack_sol = np.delete(stack_sol, (0), axis=0)
best_response = float('inf')
best_index = 0
for i in range(stack_sol.shape[0]):
    if(stack_sol[i][-1] < best_response):
        best_response = stack_sol[i][-1]
        best_index = i
print("number of roundings: ",num_of_iterations)   
print("best_response: ",best_response)   
print("\n",stack_sol[best_index])   


number of roundings:  2000
best_response:  16.0

 [ 0.  1.  0.  2.  4.  5.  6.  2.  3. 16.]


In [197]:
stack_sol

array([[ 0.,  0.,  1., ...,  3.,  6., 24.],
       [ 0.,  0.,  0., ...,  1.,  1., 26.],
       [ 0.,  0.,  0., ...,  1.,  1., 24.],
       ...,
       [ 0.,  0.,  1., ...,  2.,  2., 24.],
       [ 0.,  1.,  0., ...,  3.,  2., 22.],
       [ 0.,  0.,  1., ...,  2.,  2., 24.]])